# --- NOTE BOOK APPLICATION DETECTION FAUX BILLETS --

In [62]:
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

# --- Paramètres ---
FEATURES = ["diagonal","height_left","height_right","margin_low","margin_up","length"]
SEUIL = 0.50

# --- Entraîner sur billets.csv ---

In [65]:
train_df = pd.read_csv("billets.csv", sep=";")
train_df["is_fake"] = (~train_df["is_genuine"].astype(bool)).astype(int)

for c in FEATURES:
    train_df[c] = pd.to_numeric(train_df[c], errors="coerce")

X_train = train_df[FEATURES].values
y_train = train_df["is_fake"].values

modele_rl = Pipeline([
    ("imp",    SimpleImputer(strategy="median")),
    ("scaler", StandardScaler()),
    ("clf",    LogisticRegression(max_iter=2000, solver="liblinear", class_weight="balanced"))
]).fit(X_train, y_train)

# --- Charger les nouveaux billets ---

In [73]:
prod_df = pd.read_csv("billets_production.csv", sep=None, engine="python")
prod_df.columns = [c.strip().lower().replace(" ", "_") for c in prod_df.columns]

missing = [c for c in FEATURES if c not in prod_df.columns]
if missing:
    raise ValueError(f"Colonnes manquantes dans billets_production.csv : {missing}\n"
                     f"Colonnes attendues : {FEATURES}")

for c in FEATURES:
    prod_df[c] = pd.to_numeric(prod_df[c], errors="coerce")

#  --- Résultat ---

In [76]:
X_prod = prod_df[FEATURES].values
proba_faux = modele_rl.predict_proba(X_prod)[:, 1]
pred_bin   = (proba_faux >= SEUIL).astype(int)
label      = np.where(pred_bin == 1, "faux", "vrai")

id_col = next((c for c in ["id","id_billet","index","identifiant"] if c in prod_df.columns), None)
mini = pd.DataFrame({
    (id_col or "billet_id"): prod_df[id_col] if id_col else prod_df.index,
    "proba_faux": np.round(proba_faux, 4),
    "prediction": label
})

print("Résumé des prédictions :")
print(mini["prediction"].value_counts())
display(mini.head(10))

Résumé des prédictions :
prediction
faux    3
vrai    2
Name: count, dtype: int64


id  proba_faux prediction
0  A_1      0.9995       faux
1  A_2      0.9999       faux
2  A_3      0.9999       faux
3  A_4      0.0378       vrai
4  A_5      0.0002       vrai